<p style="float: left;"><a href="upper-type-bounds.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="lower-type-bounds.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Variance annotations

Variance is how subtyping between more complex types relates to subtyping between their components.

Suppose `A` and `B` are types, and `K<U>` denotes application of a type constructor `K` with type argument `U`. 

The kind of subtyping relationships are:

- **covariant** if `A` $\leq$ `B`, then `K<A>` $\leq$ `K<B>`.
- **contravariant** if `A` $\leq$ `B`, then `K<B>` $\leq$ `K<A>`.
- **variant** if covariant or contravariant;
- **invariant** or nonvariant if not variant.

In [ ]:
class Foo[+A] // A covariant class
class Bar[-A] // A contravariant class
class Baz[A]  // An invariant class

The combination of type parameters and subtyping poses some interesting questions. For instance, _**should Stack[String] be a subtype of Stack[AnyRef]?**_

Intuitively, this seems OK, since a stack of `String` is a special case of a stack of `AnyRef`.

```scala
class Stack[+A]
```

In a purely functional world, all types could be co-variant. However, the situation
changes once we introduce mutable data. 

Consider the case of arrays in Java.



```java
class Array[A] {
    def apply(index: Int): A
    def update(index: Int, elem: A)
}
```

In Java, arrays are covaraint.
This might seem natural but
leads to safety problems that require special _runtime checks_.

```java
val x = new Array[String](1)
val y: Array[Any] = x
y(0) = new Rational(1, 2) 
```

**We end up storing a rational number in an
array of strings**, which clearly violates type soundness.

## Scala's variance approuch

Scala solves this problem instead statically instead of in runtime.

Scala uses a conservative approximation to verify soundness of variance annotations. 
**A covariant type parameter of a class may only appear in co-variant positions inside the class**. 

The co-variant positions are:

1) the types of values in the class,
2) the result types of methods in the class,
3) and type arguments to other covariant types.

**Not co-variant are types of formal method parameters**. 

Hence, the following class definition would have been rejected:

```scala
class Array[+A] {
    def apply(index: Int): A
    def update(index: Int, elem: A)
}
```

Intuitively, the compiler was correct in rejecting the update procedure in a co-variant class because update potentially changes state, and therefore
undermines the soundness of co-variant subtyping.

However, **there are also methods which do not mutate state, but where a type parameter still appears contra-variantly**. 

An example is `push` in type `Stack`.


```scala
class Stack[+A] {
    def push(x: A): Stack[A] = . . .
}
```

This is a pity, because, **unlike arrays, stacks are purely functional data structures and
therefore should enable co-variant subtyping**. However, there is a a way to solve the
problem by using a polymorphic method with a [lower type parameter bound](lower-type-bounds.ipynb).

<p style="float: left;"><a href="upper-type-bounds.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="lower-type-bounds.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>